<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C10-Windows_Privilege_Escalation/Creating_The_Vulnerable_Blackhat_Service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Windows Privilege Escalation__*

So you've popped a box inside a nice, juicy Windows network. Maybe you leveraged a remote heap overflow, or you phished your way in. It's time to start looking for ways to escalate privileges.

Even if you're already operating as SYSTEM or Administrator, you probably want several ways of achieving those privileges, in case a patch cycle kills your access. It can be important to have a catalog of privilege escalations in your back pocket, as some enterprises run software that may be difficult to analyze in your own environment, and you may not run into that software until you're in an enterprise of the same size or composition.

In a typical privilege escalation, you'd exploit a poorly coded driver or native Windows kernel issue, but if you use a low-quality exploit or there's a problem during exploitation, you run the risk of causing system instability.
Let's explore some other means of acquiring elevated privileges on Windows. System administrators in large enterprises commonly schedule tasks or services that execute child processes, or run VBScript or PowerShell scripts to automate activities. Vendors, too, often have automated, built-in tasks that behave the same way. We'll try to take advantage of any high-privilege processes that handle files or execute binaries that are writable by low-privilege users. There are countless ways for you to try to escalate privileges on Windows, and we'll cover only a few. However, when you understand these core concepts, you can expand your scripts to begin exploring other dark, musty corners of your Windows targets.

We'll start by learning how to apply Windows Management Instrumentation (__WMI__) programming to create a flexible interface that monitors the creation of new processes. We'll harvest useful data such as the filepaths, the user who created the process, and enabled privileges. Then we'll hand off all filepaths to a file-monitoring script that continuously keeps track of any new files created, as well as what gets written to them. This tells us which files the high-privilege processes are accessing. Finally, we'll intercept the file-creation process by injecting our own scripting code into the file and make the high-privilege process execute a command shell. The beauty of this whole process is that it doesn't involve any API hooking, so we can fly under most antivirus software's radar.

### *__Installing the Prerequisites__*

We need to install a few libraries to write the tooling in this chapter. Execute the following in a __cmd.exe__ shell on Windows:

```
C:\Users\tim\work> pip install pywin32 wmi pyinstaller
```

You may have installed __pyinstaller__ when you made your keylogger and screenshot-taker in Chapter 8, but if not, install it now (you can use __pip__). Next, we'll create the sample service we'll use to test our monitoring scripts.

### *__Creating the Vulnerable Blackhat Service__*

The service we're creating emulates a set of vulnerabilities commonly found in large enterprise networks. We'll be attacking it later in this chapter. This service will periodically copy a script to a temporary directory and execute it from that directory. Open __bhservice.py__ to get started:

In [ ]:
import os
import servicemanager
import shutil
import subprocess
import sys

import win32event
import win32service
import win32serviceutil

SRCDIR = 'C:\\Users\\tim\\work'
TGTDIR = 'C:\\Windows\\TEMP'

Here, we do our imports, set the source directory for the script file, and then set the target directory where the service will run it. Now, we'll create the actual service using a class:

In [ ]:
class BHServerSvc(win32serviceutil.ServiceFramework):
    _svc_name = 'BlackHatService'
    _svc_display_name = 'Black Hat Service'
    _svc_description_ = ('Executes VBScript at regular intervals.' +
                         " What could possibly go wrong?")

    def __init__(self, args): #[1]
        self.vbs = os.path.join(TGTDIR, 'bhservice_task.vbs')
        self.timeout = 1000 * 60

        win32serviceutil.ServiceFramework.__init__(self, args)
        self.hWaitStop = win32event.CreateEvent(None, 0, 0, None)

    def SvcStop(self): #[2]
        self.ReportServiceStatus(win32service.SERVICE_STOP_PENDING)
        win32event.SetEvent(self.hWaitStop)

    def SvcDoRun(self): #[3]
        self.ReportServiceStatus(win32service.SERVICE_RUNNING)
        self.main()

This class is a skeleton of what any service must provide. It inherits from the __win32serviceutil.ServiceFramework__ and defines three methods. In the __\_\_init\_\___ method, we initialize the framework, define the location of the script to run, set a time out of one minute, and create the event object __[1]__. In the __SvcStop__ method, we set the service status and stop the service __[2]__. In the __SvcDoRun__ method, we start the service and call the __main__ method in which our tasks will run __[3]__. We define this __main__ method next:

In [ ]:
    def main(self):
        while True: #[1]
            ret_code = win32event.WaitForSingleObject(self.hWaitStop,
                                                      self.timeout)
            if ret_code == win32event.WAIT_OBJECT_0: #[2]
                servicemanager.LogInfoMsg('Service is stopping')
                break
            src = os.path.join(SRCDIR, 'bhservice_task.vbs')
            shutil.copy(src, self.vbs)
            subprocess.call('cscript.exe %s' % self.vbs, shell=False) #[3]
            os.unlink(self.vbs)

In __main__, we set up a loop __[1]__ that runs every minute, because of the __self.timeout__ parameter, until the service receives the stop signal __[2]__. While it's running, we copy the script file to the target directory, execute the script, and remove the file __[3]__.

In the main block, we handle any command line arguments:

In [ ]:
if __name__ == '__main__':
    if len(sys.argv) == 1:
        servicemanager.Initialize()
        servicemanager.PrepareToHostSingle(BHServerSvc)
        servicemanager.StartServiceCtrlDispatcher()
    else:
        win32serviceutil.HandleCommandLine(BHServerSvc)

You may sometimes want to create a real service on a victim machine. This skeleton framework gives you the outline for how to structure one. You can find the __bhservice_tasks.vbs__ script at https://nostarch.com/black-hat-python2E/. Place the file in a directory with __bhservice.py__ and change __SRCDIR__ to point to this directory. Your directory should look like this:

```
06/22/2020  09:02 AM    <DIR>           .
06/22/2020  09:02 AM    <DIR>           ..
06/22/2020  11:26 AM              2,099  bhservice.py
06/22/2020  11:08 AM              2,501  bhservice_task.vbs
```

Now create the service executable with __pyinstaller__:

```
C:\Users\tim\work> pyinstaller -F --hiddenimport win32timezone bhservice.py
```

This command saves the __bservice.exe__ file in the __dist__ subdirectory. Let's change into that directory to install the service and get it started. As Administrator, run these commands:

```
C:\Users\tim\work\dist> bhservice.exe install
C:\Users\tim\work\dist> bhservice.exe start
```

Now, every minute, the service will write the script file into a temporary directory, execute the script, and delete the file. It will do this until you run the __stop__ command.

```
C:\Users\tim\work\dist> bhservice.exe stop
```

You can start or stop the service as many times as you like. Keep in mind that if you change the code in __bhservice.py__, you'll also have to create a new executable with __pyinstaller__ and have Windows reload the service with the __bhservice update__ command. When you've finished playing around with the service in this chapter, remove it with __bhservice remove__.

You should be good to go. Now let's get on with the fun part!